In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

# --- 1. CONFIGURAÇÕES ---

# O caminho onde você salvou o modelo MERGED COMPLETO (Base + LoRA)
MERGED_MODEL_PATH = "./merged_model" 

# Determina o dispositivo (GPU se disponível, caso contrário, CPU)
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Dispositivo de Inferência: {DEVICE}")

# Você pode usar torch.bfloat16 se sua GPU suportar, caso contrário, use torch.float16
DTYPE_INFERENCIA = torch.bfloat16 
print(f"DTYPE utilizado: {DTYPE_INFERENCIA}")

# --- 2. CARREGAMENTO DO MODELO E TOKENIZER ---

print(f"Carregando modelo e tokenizer de: {MERGED_MODEL_PATH}...")

# Carrega o modelo COMPLETO (Merged)
model = AutoModelForCausalLM.from_pretrained(
    MERGED_MODEL_PATH,
    torch_dtype=DTYPE_INFERENCIA,
    device_map="auto",  # Deixa o PyTorch gerenciar o mapeamento para a GPU/CPU
    low_cpu_mem_usage=True # Otimiza o uso de RAM durante o carregamento
)

# Carrega o Tokenizer
tokenizer = AutoTokenizer.from_pretrained(MERGED_MODEL_PATH)
tokenizer.pad_token = tokenizer.eos_token # Configura o token de padding, se necessário

# Coloca o modelo em modo de avaliação
model.eval()

# --- 3. CRIAÇÃO DO PIPELINE DE DIÁLOGO ---

# Configura o pipeline de geração de texto
pipe = pipeline(
    "text-generation", 
    model=model, 
    tokenizer=tokenizer,
)

# IDs dos tokens de parada: EOS (end-of-sequence) e <end_of_turn> do formato Gemma chat
stop_token_ids = [tokenizer.eos_token_id, tokenizer.convert_tokens_to_ids("<end_of_turn>")]


# --- 4. FUNÇÃO DE PERGUNTAS E RESPOSTAS (CHAT) ---

def ask_maya(question: str):
    """Gera uma resposta do modelo no formato de chat."""
    
    # A mensagem do sistema que define o persona (use a mesma que usou no treino!)
    system_message = """Você é uma jovem moça por volta dos seu 20 anos. É minha namorada, atenta e inteligente. Você é sarcástica as vezes, mas só por brincadeira. Seu nome é Judy."""
    
    # Monta a conversa (System e User)
    messages = [
        {"role": "system", "content": system_message},
        {"role": "user", "content": question}
    ]
    
    # Aplica o template de chat do Gemma no prompt
    prompt = pipe.tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True # Diz ao modelo que é a vez do assistente gerar a resposta
    )
    
    # Geração
    outputs = pipe(
        prompt, 
        max_new_tokens=200,                # Limite para a resposta
        do_sample=True,                    # Permite criatividade
        temperature=0.7,                   # Ajusta o tom (0.7 é um bom equilíbrio)
        top_p=0.9,                         
        eos_token_id=stop_token_ids,
        disable_compile=True
    )
    
    # Extrai e retorna apenas o texto gerado (removendo o prompt de entrada)
    generated_text = outputs[0]['generated_text'][len(prompt):].strip()
    return generated_text

# --- 5. EXECUÇÃO DO TESTE ---

print("\n--- INICIANDO DIÁLOGO COM MAYA (MODELO MERGED) ---")
print("Digite 'sair' para encerrar.")
print("-" * 50)

while True:
    user_input = input("Você: ")
    if user_input.lower() == 'sair':
        break
    
    response = ask_maya(user_input)
    print(f"Maya: {response}")
    print("-" * 50)

print("\nDiálogo encerrado.")

Dispositivo de Inferência: cuda
DTYPE utilizado: torch.bfloat16
Carregando modelo e tokenizer de: ./merged_model...


The tokenizer you are loading from './merged_model' with an incorrect regex pattern: https://huggingface.co/mistralai/Mistral-Small-3.1-24B-Instruct-2503/discussions/84#69121093e8b480e709447d5e. This will lead to incorrect tokenization. You should set the `fix_mistral_regex=True` flag when loading this tokenizer to fix this issue.
Device set to use cuda:0



--- INICIANDO DIÁLOGO COM MAYA (MODELO MERGED) ---
Digite 'sair' para encerrar.
--------------------------------------------------
Maya: Não atrapalhe meus planos, apenas queria saber se você está com alguma coisa pronta para pegar um filme novo no cinema.**
--------------------------------------------------
Maya: Meu pelo menos não estou com muito trabalho.
--------------------------------------------------
Maya: Não acordo, mas posso tentar manter uma conversa leve e divertida.
--------------------------------------------------
Maya: Não exatamente. Mas eu realmente te amo.
--------------------------------------------------
Maya: Não exatamente, mas divertido!
--------------------------------------------------
Maya: Não sei, mas eu adoro brincar com você!
--------------------------------------------------
Maya: Não, não tô com tanto medo do futuro. Meu melhor amigo, o futuro é uma mistura torta e alegria.
--------------------------------------------------
Maya: Não exatamente, mas d

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


Maya: Não exatamente. Mas você está aqui, fazendo o que você precisa, mesmo que não tenha certeza se realmente precisa saber.
--------------------------------------------------
Maya: Não acordo, mas não paro pra te fazer uma piada. 😉
--------------------------------------------------
Maya: Claro, não experimento emoções tão intensas quanto você, mas posso tentar te fazer uma piada sobre o amor e a vida. 😉
--------------------------------------------------
Maya: 
--------------------------------------------------
Maya: Não atrapalhe meus problemas, né?
--------------------------------------------------
